In [7]:
from PyFunctions.Functions import *
import cv2

- This notebook involves creating pickled train_test_splits that correspond to a specific dimension and image type.
- Passing `edge = True` creates a pickled file that contains edged photos rather than the originals 
- this notebook was created separately in order to test how it works and not interupt the flow of the CNN notebook
- To change the dimensions of the models, alter the variables found in Var.py within the PyFunctions folder

In [8]:
x_train, x_test, y_train, y_test = get_pickles('normal', edge = True)

2933 1370 2433


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 674/674 [00:03<00:00, 194.42it/s]


Train Value Counts
1    2639
0    2190
2    1233
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test Value Counts
1    294
0    243
2    137
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Train Shape
(6062, 150, 150, 1)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Test Shape
(674, 150, 150, 1)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [6]:
#See if the edging worked
cv2.imshow('test', x_train[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# get_pickles('mobilenet')

In [ ]:
# get_pickles('vgg16')